# Generate Test Results

## Prepare Test Images

In [1]:
from utils.prepare_images import prepare_images
import os
import pandas as pd

%load_ext autoreload
%autoreload 2

In [23]:
# put videos here!
#video_base = 'data/videos'
video_base = r'E:/e6691/6691_assignment2/videos'
videos = [vid for vid in os.listdir(video_base) if vid.endswith('mp4')]
# images will be output to here
#image_base = 'data/images'
image_base = r'E:/e6691/6691_assignment2/images'

In [16]:
len(videos)

60

In [17]:
test_videos = sorted(videos)
len(test_videos)

60

In [20]:
# command line: 
# ffmpeg -i {input_video} -r {frame_rate} [-f {force_format} (not needed)] {output_images}
# doc: https://ffmpeg.org/ffmpeg.html
for video in test_videos:
    input_path = os.path.join(video_base, video)
    # make dirs
    output_base = image_base + '\{}'.format(video.split('.')[0])
    if not os.path.exists(output_base):
        os.mkdir(output_base)
    output_path = os.path.join(output_base, '%d.png')
    # # command
    print('Frames extracted from {} to {}'.format(input_path, output_path))
    # !!!!!! BECAREFUL WITH THIS LINE !!!!!!
    !ffmpeg -i {input_path} -r 1 {output_path}

Frames extracted from E:\e6691\6691_assignment2\videos\RALIHR_surgeon01_fps01_0071.mp4 to E:\e6691\6691_assignment2\images\RALIHR_surgeon01_fps01_0071\%d.png
Frames extracted from E:\e6691\6691_assignment2\videos\RALIHR_surgeon01_fps01_0072.mp4 to E:\e6691\6691_assignment2\images\RALIHR_surgeon01_fps01_0072\%d.png


ffmpeg version 5.0-essentials_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 11.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb --enable-l

Frames extracted from E:\e6691\6691_assignment2\videos\RALIHR_surgeon01_fps01_0073.mp4 to E:\e6691\6691_assignment2\images\RALIHR_surgeon01_fps01_0073\%d.png


ffmpeg version 5.0-essentials_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 11.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb --enable-l

Frames extracted from E:\e6691\6691_assignment2\videos\RALIHR_surgeon01_fps01_0074.mp4 to E:\e6691\6691_assignment2\images\RALIHR_surgeon01_fps01_0074\%d.png


...

ffmpeg version 5.0-essentials_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 11.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb --enable-l

Frames extracted from E:\e6691\6691_assignment2\videos\RALIHR_surgeon02_fps01_0004.mp4 to E:\e6691\6691_assignment2\images\RALIHR_surgeon02_fps01_0004\%d.png


ffmpeg version 5.0-essentials_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 11.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb --enable-l

Frames extracted from E:\e6691\6691_assignment2\videos\RALIHR_surgeon03_fps01_0001.mp4 to E:\e6691\6691_assignment2\images\RALIHR_surgeon03_fps01_0001\%d.png


ffmpeg version 5.0-essentials_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 11.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb --enable-l

In [21]:
labels_path = 'data/labels/video.phase.trainingData.clean.StudentVersion.csv'
names_path = 'data/labels/all_labels_hernia.csv'
labels_df = pd.read_csv(labels_path)
names_df = pd.read_csv(names_path, index_col='id')

In [27]:
# all labels are set to 14
prepare_images(test_videos, image_base, labels_df, names_df)

## Predict

In [28]:
from models.SVRCNet.svrc import SVRC
from utils.trainer import Evaluator
from utils.read_videos import read

import torch
from torchvision import transforms

print(torch.version.cuda)

11.3


In [29]:
image_paths, labels = read(test_videos, image_base, ind_end=60)

In [30]:
X_test = sum(image_paths, [])
y_test = sum(labels, [])

In [31]:
list(zip(X_test, y_test))

[('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/1-14.png', 14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/2-14.png', 14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/3-14.png', 14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/4-14.png', 14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/5-14.png', 14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/6-14.png', 14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/7-14.png', 14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/8-14.png', 14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/9-14.png', 14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/10-14.png',
  14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/11-14.png',
  14),
 ('E:/e6691/6691_assignment2/images/RALIHR_surgeon01_fps01_0071/12-14.png',
  14),
 ('E:/e6691/6691_assignment2/im

In [32]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = SVRC()
if torch.cuda.is_available():
    model.to(device)

# this does not necessarily affect what mode we're running at
# weights of resnet is not changed anyway
# !!!!!! DELETE THIS LINE !!!!!!
WeightsPath_LSTM = 'models/weights_resnet18_50_1'
model.load_state_dict(torch.load(WeightsPath_LSTM, map_location=device))

<All keys matched successfully>

In [33]:
data_transform = {
    "train": transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])
}

In [34]:
evaluator = Evaluator(model, device)

In [35]:
preds_res = evaluator.predict(X_test, data_transform['train'], pretrain=True)

In [36]:
preds_res

[tensor([13, 13, 13], device='cuda:0'),
 tensor([13, 13, 13], device='cuda:0'),
 tensor([13, 13, 13], device='cuda:0'),
 tensor([13, 13, 13], device='cuda:0'),
 tensor([13,  7,  7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([ 7,  7, 13], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 8], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], device='cuda:0'),
 tensor([7, 7, 7], d

In [40]:
preds_lstm = evaluator.predict(X_test, data_transform['train'], pretrain=False)

In [41]:
# seems that lstm does do something in terms of smoothing the predictions
list(zip(preds_res, preds_lstm))

[(13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (13, 13),
 (6, 6),
 (6, 6),
 (13, 13),
 (6, 13),
 (9, 13),
 (9, 13),
 (9, 11),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (9, 6),
 (9, 6),
 (9, 11),
 (6, 6),
 (9, 11),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (13, 6),
 (13, 13),
 (13, 13),
 (13, 13),
 (9, 13),
 (6, 6),
 (6, 13),
 (6, 6),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (13, 13),
 (6, 13),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (13, 6),
 (13, 6),
 (13, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6, 6),
 (6,

In [73]:
from scipy.signal import medfilt

In [59]:
evaluator.eval(preds_res, y_test)
evaluator.eval(preds_lstm, y_test)
evaluator.eval(preds_res_filtered, y_test)

Accuracy: 0.15884928080050031
Accuracy: 0.16416510318949343
Accuracy: 0.16447779862414008


0.16447779862414008

## Convert to csv

In [37]:
!pip install parse

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=4780874aabb792994f65999388a4594993f217dedce33067d53b66ea2d41242e
  Stored in directory: c:\users\29764\appdata\local\pip\cache\wheels\d6\9c\58\ee3ba36897e890f3ad81e9b730791a153fce20caa4a8a474df
Successfully built parse


In [38]:
from parse import parse

In [39]:
template_path = 'data/labels/kaggle_template.csv'

In [68]:
template_df = pd.read_csv(template_path)
template_df

,Id,Predicted
0,001-0071-00001,NaN
1,001-0071-00002,NaN
2,001-0071-00003,NaN
3,001-0071-00004,NaN
4,001-0071-00005,NaN
...,...,...
179863,003-0001-05902,NaN
179864,003-0001-05903,NaN
179865,003-0001-05904,NaN
179866,003-0001-05905,NaN


In [41]:
parse('{}/RALIHR_surgeon{}_fps01_{}/{}-{}.png', X_test[0])[1:4]

('01', '0071', '1')

In [42]:
def format_path(s):
    ss = parse('{}/RALIHR_surgeon{}_fps01_{}/{}-{}.png', s)[1:4]
    return '{}-{}-{}'.format(
        ss[0].rjust(3, '0'), 
        ss[1].rjust(4, '0'), 
        ss[2].rjust(5, '0')
    )

In [51]:
preds_res_flatten = sum(list(map(torch.Tensor.tolist, preds_res)), [])
preds_res_flatten

[13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 13,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 9,
 7,
 7,
 7,
 9,
 9,
 9,
 9,
 9,
 7,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 13,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 

In [54]:
names_df.loc[preds_res_flatten[:5]].values.flatten()

array(['transitionary idle', 'transitionary idle', 'transitionary idle',
       'transitionary idle', 'transitionary idle'], dtype=object)

In [56]:
len(X_test), len(preds_res), len(preds_res_flatten)

(180131, 60043, 180129)

In [58]:
preds_res_flatten += [13, 13]

In [80]:
preds_res_filtered = medfilt(preds_res_flatten, 5)
preds_res_filtered

array([13, 13, 13, ..., 13, 13, 13])

In [81]:
test_df = pd.DataFrame({
    'preds': names_df.loc[preds_res_filtered].values.flatten()
}, index=list(map(format_path, X_test)))
test_df

,preds
001-0071-00001,transitionary idle
001-0071-00002,transitionary idle
001-0071-00003,transitionary idle
001-0071-00004,transitionary idle
001-0071-00005,transitionary idle
...,...
003-0001-05902,transitionary idle
003-0001-05903,transitionary idle
003-0001-05904,transitionary idle
003-0001-05905,transitionary idle


In [82]:
template_df['Id']

0         001-0071-00001
1         001-0071-00002
2         001-0071-00003
3         001-0071-00004
4         001-0071-00005
               ...      
179863    003-0001-05902
179864    003-0001-05903
179865    003-0001-05904
179866    003-0001-05905
179867    003-0001-05906
Name: Id, Length: 179868, dtype: object

In [83]:
test_df.index

Index(['001-0071-00001', '001-0071-00002', '001-0071-00003', '001-0071-00004',
       '001-0071-00005', '001-0071-00006', '001-0071-00007', '001-0071-00008',
       '001-0071-00009', '001-0071-00010',
       ...
       '003-0001-05897', '003-0001-05898', '003-0001-05899', '003-0001-05900',
       '003-0001-05901', '003-0001-05902', '003-0001-05903', '003-0001-05904',
       '003-0001-05905', '003-0001-05906'],
      dtype='object', length=180131)

In [84]:
# select test_df with template
preds_df = test_df.loc[template_df['Id']]
preds_df

,preds
001-0071-00001,transitionary idle
001-0071-00002,transitionary idle
001-0071-00003,transitionary idle
001-0071-00004,transitionary idle
001-0071-00005,transitionary idle
...,...
003-0001-05902,transitionary idle
003-0001-05903,transitionary idle
003-0001-05904,transitionary idle
003-0001-05905,transitionary idle


In [85]:
preds_df.to_csv('data/labels/kaggle_preds.csv')